In [1]:
import pandas as pd

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
data = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
data.head()

,preco,vendido,idade_do_modelo,km_por_ano
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [2]:
rename = {
    "preco": "price",
    "vendido": "sold",
    "idade_do_modelo": "model_age",
    "km_por_ano": "km_per_year"
}

data = data.rename(columns=rename)
data.head()

,price,sold,model_age,km_per_year
0,30941.02,1,18,35085.22134
1,40557.96,1,20,12622.05362
2,89627.50,0,12,11440.79806
3,95276.14,0,3,43167.32682
4,117384.68,1,4,12770.11290


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from  sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = data[["price", "model_age", "km_per_year"]]
y = data["sold"]

SEED  = 158020
np.random.seed(SEED)
train_x, test_x, train_y, test_y = train_test_split(x, y, 
                                                    test_size=0.25,
                                                    stratify=y)

print(f"We will train with {len(train_x)} elements and test with {len(test_x)} elements.")

We will train with 7500 elements and test with 2500 elements.


In [4]:
from  sklearn.dummy  import DummyClassifier

dummy_stratified = DummyClassifier(strategy="stratified")
dummy_stratified.fit(train_x, train_y)
accuracy = dummy_stratified.score(test_x,test_y) * 100

print(f"The accuracy of the dummy stratified model was:  {accuracy:.2f}%")

The accuracy of the dummy stratified model was:  50.96%


In [5]:
from sklearn.tree import DecisionTreeClassifier

SEED  = 158020
np.random.seed(SEED)
model = DecisionTreeClassifier(max_depth=2)
model.fit(train_x,  train_y)
predictions = model.predict(test_x)

accuracy = accuracy_score(test_y, predictions) * 100
print(f"The accuracy was: {accuracy:.2f}%")

The accuracy was: 71.92%


In [6]:
x = data[["price", "model_age", "km_per_year"]]
y = data["sold"]

SEED  = 158020
np.random.seed(SEED)
train_x, test_x, train_y, test_y = train_test_split(x, y, 
                                                    test_size=0.25,
                                                    stratify=y)

print(f"We will train with {len(train_x)} elements and test with {len(test_x)} elements.")

model = DecisionTreeClassifier(max_depth=2)
model.fit(train_x,  train_y)
predictions = model.predict(test_x)

accuracy = accuracy_score(test_y, predictions) * 100
print(f"The accuracy was: {accuracy:.2f}%")

We will train with 7500 elements and test with 2500 elements.
The accuracy was: 71.92%



# Using and evaluating with cross validation

In [7]:
from sklearn.model_selection import cross_validate

SEED  = 301
np.random.seed(SEED)

model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv=3, return_train_score=False)
mean = results["test_score"].mean()
std = results["test_score"].std()
print(f"Accuracy with cross-validation, 3 = [{(mean - 2*std)*100:.2f}, {(mean + 2*std)*100:.2f}]")

Accuracy with cross-validation, 3 = [74.99, 76.57]


In [8]:
SEED  = 301
np.random.seed(SEED)

model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv=10, return_train_score=False)
mean = results["test_score"].mean()
std = results["test_score"].std()
print(f"Accuracy with cross-validation, 10 = [{(mean - 2*std)*100:.2f}, {(mean + 2*std)*100:.2f}]")

Accuracy with cross-validation, 10 = [74.24, 77.32]


In [9]:
SEED  = 301
np.random.seed(SEED)

model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv=5, return_train_score=False)
mean = results["test_score"].mean()
std = results["test_score"].std()
print(f"Accuracy with cross validate, 5 = [{(mean - 2*std)*100:.2f}, {(mean + 2*std)*100:.2f}]")

Accuracy with cross validate, 5 = [75.21, 76.35]


# Randomness in cross validate

In [10]:
def print_results(results):
    mean = results["test_score"].mean()
    std = results["test_score"].std()
    print(f"Mean accuracy: {mean*100:.2f}")
    print(f"Accuracy interval: [{(mean - 2*std)*100:.2f}, {(mean + 2*std)*100:.2f}]")

In [11]:
from sklearn.model_selection import KFold

SEED  = 301
np.random.seed(SEED)

cv = KFold(n_splits=10)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv=cv, return_train_score=False)
print_results(results)


Mean accuracy: 75.78
Accuracy interval: [74.37, 77.19]


In [12]:
SEED  = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x, y, cv=cv, return_train_score=False)
print_results(results)

Mean accuracy: 75.76
Accuracy interval: [73.26, 78.26]


# Simulate a horribly unlucky situation
Could be a situation of 'unluck' or an imbalanced ratio of examples between the classes

In [13]:
data_unluck = data.sort_values("sold", ascending=True)
x_unluck = data_unluck[["price", "model_age", "km_per_year"]]
y_unluck = data_unluck["sold"]
data_unluck.head()

,price,sold,model_age,km_per_year
10,75668.42,0,2,22635.36710
7,65265.09,0,10,18666.73466
3,95276.14,0,3,43167.32682
2,89627.50,0,12,11440.79806
4221,64724.65,0,10,13194.97866


In [14]:
from sklearn.model_selection import KFold

SEED  = 301
np.random.seed(SEED)

cv = KFold(n_splits=10)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x_unluck, y_unluck, cv=cv, return_train_score=False)
print_results(results)

Mean accuracy: 57.96
Accuracy interval: [34.56, 81.36]


In [15]:
SEED  = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x_unluck, y_unluck, cv=cv, return_train_score=False)
print_results(results)

Mean accuracy: 75.78
Accuracy interval: [73.88, 77.68]


In [16]:
from sklearn.model_selection import StratifiedKFold

SEED  = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits=10, shuffle=True)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x_unluck, y_unluck, cv=cv, return_train_score=False)
print_results(results)

Mean accuracy: 75.78
Accuracy interval: [72.91, 78.65]


# Generating random data

In [17]:
np.random.seed(SEED)
data["model"] = data.model_age + np.random.randint(-2, 3, size=10000)
data.model = data.model + abs(data.model.min()) + 1
data.head()

,price,sold,model_age,km_per_year,model
0,30941.02,1,18,35085.22134,18
1,40557.96,1,20,12622.05362,24
2,89627.50,0,12,11440.79806,14
3,95276.14,0,3,43167.32682,6
4,117384.68,1,4,12770.11290,5


# Cross validation using groups

In [18]:
from sklearn.model_selection import GroupKFold

SEED  = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
model = DecisionTreeClassifier(max_depth=2)
results = cross_validate(model, x_unluck, y_unluck, cv=cv, groups = data.model, return_train_score=False)
print_results(results)

Mean accuracy: 75.79
Accuracy interval: [74.03, 77.56]


# Cross validation with StandardScaler

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(train_x)
train_x_scaled = scaler.transform(train_x)
test_x_scaled = scaler.transform(test_x)

model = SVC()
model.fit(train_x_scaled, train_y)
predictions = model.predict(test_x_scaled)

accuracy = accuracy_score(test_y, predictions) * 100
print(f"The accuracy was: {accuracy:.2f}%")

The accuracy was: 74.40%


In [20]:
from sklearn.model_selection import GroupKFold

SEED  = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
model = SVC()
results = cross_validate(model, x_unluck, y_unluck, cv=cv, groups = data.model, return_train_score=False)
print_results(results)

Mean accuracy: 77.21
Accuracy interval: [74.73, 79.69]


In [21]:
scaler = StandardScaler()
scaler.fit(x_unluck)
x_unluck_scaled = scaler.transform(x_unluck)


In [22]:
from sklearn.model_selection import GroupKFold

SEED  = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
model = SVC()
results = cross_validate(model, x_unluck_scaled, y_unluck, cv=cv, groups = data.model, return_train_score=False)
print_results(results)

Mean accuracy: 76.63
Accuracy interval: [74.07, 79.19]


In [23]:
from sklearn.pipeline import Pipeline

SEED  = 301
np.random.seed(SEED)

scaler = StandardScaler()
model = SVC()

pipeline = Pipeline([('transformation',scaler), ('estimator', model)])

cv = GroupKFold(n_splits=10)
results = cross_validate(pipeline, x_unluck, y_unluck, cv=cv, groups = data.model, return_train_score=False)
print_results(results)

Mean accuracy: 76.62
Accuracy interval: [74.02, 79.22]
